# Filter Data Komentar YouTube

Notebook ini melakukan filtering data komentar untuk membuang:
1. **Noise** - spam, "wkwk", "hadir gan", dll
2. **Tidak Relevan** - komentar di luar konteks topik

**Output**: Data yang relevan dan bersih untuk preprocessing lebih lanjut

In [31]:
# Import libraries
import pandas as pd
import re

# Load data
df = pd.read_csv('raw-scrape-yt.csv')
print(f"Total komentar: {len(df)}")
print("\nContoh data awal:")
df.head()

Total komentar: 2196

Contoh data awal:


,Video_ID,Teks_Komentar
0,MIo4tGN11j0,"berdoa tidak ada comment judol, amin."
1,MIo4tGN11j0,kereen.
2,MIo4tGN11j0,"Sempat mikir mau pindah ke negara sebelah, nge..."
3,MIo4tGN11j0,"Kalo kabur mau kemana ke Singapur ,emang di Si..."
4,MIo4tGN11j0,Jg ke malaysia.....pergi sono india bangla nep...


## 1. Filter Noise

Membuang komentar yang merupakan noise murni (spam, ketawa doang, terlalu pendek, dll)

In [32]:
def filter_noise(teks):
    """
    Return True jika komentar VALID (BUKAN noise).
    Return False jika komentar adalah NOISE (spam, pendek, dll).
    """
    if pd.isna(teks) or len(teks.strip()) < 3:
        return False
    
    teks_lower = teks.lower().strip()
    
    # Daftar pola noise yang harus dibuang
    noise_patterns = [
        r'^[@#]',  # Dimulai dengan @ atau #
        r'^\d+$',  # Hanya angka
        r'^[^\w\s]+$',  # Hanya simbol/emoji
        r'(subscribe|subs|subrek|gw klik subs)',  # Promosi subscribe
        r'(like|liek|liak|laik)\s*(dulu|dong|yuk)',  # Ajakan like
        r'^(wkwk|haha|xixi|kwkw|wkwkwk|hahaha)+$',  # Hanya ketawa
        r'^(amin|amiin|aamiin)+$',  # Hanya amin
        r'^(first)',  # Klaim urutan
        r'(giveaway|give away|kontes)',  # Promo
        r'(qna|q&a|q n a|q & a)',
        r'^(setuju|tidak setuju|ngga setuju)+$' #hanya setuju atau tidak setuju
    ]
    
    for pattern in noise_patterns:
        if re.search(pattern, teks_lower):
            return False
    
    return True

# --- Terapkan Filter Tahap 1 ---
print(f"Komentar sebelum filter: {len(df)}")

df_tahap_1 = df[df['Teks_Komentar'].apply(filter_noise)].copy()

print(f"Komentar setelah filter noise: {len(df_tahap_1)}")
print(f"Komentar noise terbuang: {len(df) - len(df_tahap_1)}")

Komentar sebelum filter: 2196
Komentar setelah filter noise: 2086
Komentar noise terbuang: 110


## 2. Filter Konteks

Membuang komentar yang tidak relevan dengan topik (strategi LONGGAR: minimal ada 1 keyword)

In [33]:
KEYWORD = [
    'kabur', 'pindah', 'negara', 'leave indo', 'pajak', 'gaji', 'umr',
    'biaya hidup', 'korupsi', 'sandwich', 'luar negeri',
    'paspor', 'warga negara', 'singapur', 'singapore', 'australia', 'aussie', 'jepang', 'eropa', 'wni',
    'capek', 'lelah', 'pemerintah', 'mending', 'percuma', 'beban',
    'suram', 'males', 'ga jelas', 'ga ada harapan', 'nyicil', 'susah',
    'setuju', 'sulit', 'stres', 'politik', 'birokrasi', 'konoha'
]

print(f"\nTotal keyword unik untuk filter konteks: {len(KEYWORD)}")

def filter_konteks(teks):
    """
    Return True jika komentar RELEVAN (Strategi LONGGAR dan Efisien).
    Return False jika di luar konteks.
    """
    teks_lower = str(teks).lower()
    
    # Cukup cek satu kali terhadap list gabungan
    if any(kata in teks_lower for kata in KEYWORD):
        return True # Lolos jika mengandung SALAH SATU
        
    return False

# --- Terapkan Filter Tahap 2 ---
df_filtered = df_tahap_1[df_tahap_1['Teks_Komentar'].apply(filter_konteks)].copy()

print(f"Komentar setelah filter konteks: {len(df_filtered)}")
print(f"Komentar tidak relevan terbuang: {len(df_tahap_1) - len(df_filtered)}")


print("\n--- Hasil Filter Konteks ---")
print(f"Total komentar awal: {len(df)}")
print(f"Total komentar final (bersih & relevan): {len(df_filtered)}")
print(f"Total komentar terbuang (noise + tdk relevan): {len(df) - len(df_filtered)}")


Total keyword unik untuk filter konteks: 38
Komentar setelah filter konteks: 778
Komentar tidak relevan terbuang: 1308

--- Hasil Filter Konteks ---
Total komentar awal: 2196
Total komentar final (bersih & relevan): 778
Total komentar terbuang (noise + tdk relevan): 1418


## 3. Lihat Contoh Hasil Filter

In [34]:
# Tampilkan beberapa contoh komentar yang lolos filter
print("="*80)
print("CONTOH KOMENTAR YANG LOLOS FILTER")
print("="*80)

for i in range(min(10, len(df_filtered))):
    print(f"\n[{i+1}] {df_filtered['Teks_Komentar'].iloc[i][:100]}...")

print("\n" + "="*80)

CONTOH KOMENTAR YANG LOLOS FILTER

[1] Sempat mikir mau pindah ke negara sebelah, ngeapply citizenship. Tapi yah, aku cinta negara dan hara...

[2] Kalo kabur mau kemana ke Singapur ,emang di Singapur tinggal dimana rakyat Singapur aja nggak punya ...

[3] Klo sudah gelap susah terangnya lebih baik bubar...

[4] Pengen kabur tapi gak punya uang😭...

[5] Kalau mau pindah ke luar negeri ya silahkan saja...kenapa mesti koar koar ya. Kagak ngatri ane......

[6] Arab Saudi investasi di Amerika 10.000 triliun sedangkan di negara ini hanya 50 miliar🤣...

[7] 1 hal orang" bagi yg sudah diluar negeri semuanya diujung pemikirannya pasti pengen balik ke Indones...

[8] Saya udah kabur 30 tahun dari indonesia, yang disayang kan indonesia memegang sistem 1 kewarganegara...

[9] YANG PASTI SUDAH TIDAK NASIONALIS ITU ADALAH PARA WAKIL RAKYAT DAN PARA BIROKRAT NEGERI KITA. APA AL...

[10] Mau tapi.. Bukan gx bisa,bukan gx mampu Tapi gax "boleh" Emg gini di +62...  Kalo mau naik derajat m...



In [35]:
# 1. Komentar yang dianggap noise
df_noise = df[~df.index.isin(df_tahap_1.index)]
print("="*80)
print("CONTOH KOMENTAR NOISE YANG DIBUANG")
print("="*80)
for i in range(min(5, len(df_noise))):
    print(f"[{i+1}] {df_noise['Teks_Komentar'].iloc[i]}")

# 2. Komentar yang dianggap tidak relevan (lolos noise, tapi tidak lolos konteks)
df_off_topic = df_tahap_1[~df_tahap_1.index.isin(df_filtered.index)]
print("\n" + "="*80)
print("CONTOH KOMENTAR TIDAK RELEVAN YANG DIBUANG")
print("="*80)
for i in range(min(5, len(df_off_topic))):
    print(f"[{i+1}] {df_off_topic['Teks_Komentar'].iloc[i]}")
print("\n" + "="*80)

CONTOH KOMENTAR NOISE YANG DIBUANG
[1] #kokori....
[2] #indonesiasakitt
[3] 🎉🎉🎉😢😢😢😢😢
[4] 🎉🎉🎉
[5] #kaburduluaja ganti dengan #hijrahduluaja,

CONTOH KOMENTAR TIDAK RELEVAN YANG DIBUANG
[1] berdoa tidak ada comment judol, amin.
[2] kereen.
[3] Jg ke malaysia.....pergi sono india bangla nepal serumpun....rupiah rupie
[4] gue mulai demen nih org
[5] Siap siap ente motivator Indonesia gelap...dapet cipratan Marcela Santoso atau engga



## 4. Simpan Hasil Filter

Menyimpan data yang sudah difilter untuk digunakan di preprocessing

In [36]:
# Simpan hasil filter
output_file = 'data_filtered.csv'
df_filtered.to_csv(output_file, index=False, encoding='utf-8')

print("="*80)
print("✅ FILTER SELESAI!")
print("="*80)
print(f"\n📊 RINGKASAN:")
print(f"   Total komentar awal          : {len(df):,}")
print(f"   Setelah filter noise         : {len(df_tahap_1):,}")
print(f"   Setelah filter konteks       : {len(df_filtered):,}")
print(f"   Total terbuang               : {len(df) - len(df_filtered):,}")
print(f"   Persentase data valid        : {(len(df_filtered) / len(df) * 100):.2f}%")
print(f"\n💾 Data tersimpan di: {output_file}")
print(f"\n📝 Data ini siap untuk preprocessing di notebook 'preprocessing.ipynb'")
print("="*80)

✅ FILTER SELESAI!

📊 RINGKASAN:
   Total komentar awal          : 2,196
   Setelah filter noise         : 2,086
   Setelah filter konteks       : 778
   Total terbuang               : 1,418
   Persentase data valid        : 35.43%

💾 Data tersimpan di: data_filtered.csv

📝 Data ini siap untuk preprocessing di notebook 'preprocessing.ipynb'
